In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import zipfile
import os


In [2]:
# URL of the page containing the table
url = "https://forecastingdata.org/"

# Send a GET request to fetch the page content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Locate the table containing the datasets
table = soup.find_all('table')[0]  # Assuming the first table is the target

# Extract table headers
headers = [th.text.strip() for th in table.find_all('th')]
headers.append('Download Links')

# Extract table rows
rows = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    cols = [td.text.strip() for td in row.find_all('td')]
    download_links = [a['href'] for a in row.find_all('a', href=True)]
    cols.append(download_links)
    rows.append(cols)

# Create a pandas DataFrame
df = pd.DataFrame(rows, columns=headers)
df['Download'] = df["Download"].apply(lambda x: x.split('\n'))
df["Download Links"] = df["Download Links"].apply(lambda x: [l for l in x if 'zenodo' in l])
df = df.explode(["Download", "Download Links"]).reset_index()

In [3]:
df

,index,Dataset,Domain,No: of Series,Min. Length,Max. Length,Competition,Multivariate,Download,Source,Download Links
0,0,M1,Multiple,1001,15,150,Yes,No,Yearly,"Makridakis et al., 1982",https://zenodo.org/record/4656193
1,0,M1,Multiple,1001,15,150,Yes,No,Quarterly,"Makridakis et al., 1982",https://zenodo.org/record/4656154
2,0,M1,Multiple,1001,15,150,Yes,No,Monthly,"Makridakis et al., 1982",https://zenodo.org/record/4656159
3,1,M3,Multiple,3003,20,144,Yes,No,Yearly,"Makridakis and Hibon, 2000",https://zenodo.org/record/4656222
4,1,M3,Multiple,3003,20,144,Yes,No,Quarterly,"Makridakis and Hibon, 2000",https://zenodo.org/record/4656262
5,1,M3,Multiple,3003,20,144,Yes,No,Monthly,"Makridakis and Hibon, 2000",https://zenodo.org/record/4656298
6,1,M3,Multiple,3003,20,144,Yes,No,Other,"Makridakis and Hibon, 2000",https://zenodo.org/record/4656335
7,2,M4,Multiple,100000,19,9933,Yes,No,Yearly,"Makridakis et al., 2020",https://zenodo.org/record/4656379
8,2,M4,Multiple,100000,19,9933,Yes,No,Quarterly,"Makridakis et al., 2020",https://zenodo.org/record/4656410
9,2,M4,Multiple,100000,19,9933,Yes,No,Monthly,"Makridakis et al., 2020",https://zenodo.org/record/4656480


In [16]:
def download_link_extract(url):
    try:
        # Send a GET request to the page
        response = requests.get(url)
        response.raise_for_status()  # Ensure we notice bad responses
    
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")
    
        # Find all links on the page
        links = soup.find_all('a', href=True)
    
        # Look for the specific download link based on the "download=1" parameter
        download_link = None
        for link in links:
            if "download=1" in link['href']:
                download_link = link['href']
                break
    
        if download_link:
            print("Download link found:", download_link)
            return  'https://zenodo.org' + download_link
        else:
            print("Download link not found.")
    except requests.exceptions.RequestException as e:
        print("An error occurred while making the request:", e)

In [17]:
df["download_link"] = df['Download Links'].apply(lambda x: download_link_extract(x))

Download link found: /records/4656193/files/m1_yearly_dataset.zip?download=1
Download link found: /records/4656154/files/m1_quarterly_dataset.zip?download=1
Download link found: /records/4656159/files/m1_monthly_dataset.zip?download=1
Download link found: /records/4656222/files/m3_yearly_dataset.zip?download=1
Download link found: /records/4656262/files/m3_quarterly_dataset.zip?download=1
Download link found: /records/4656298/files/m3_monthly_dataset.zip?download=1
Download link found: /records/4656335/files/m3_other_dataset.zip?download=1
Download link found: /records/4656379/files/m4_yearly_dataset.zip?download=1
Download link found: /records/4656410/files/m4_quarterly_dataset.zip?download=1
Download link found: /records/4656480/files/m4_monthly_dataset.zip?download=1
Download link found: /records/4656522/files/m4_weekly_dataset.zip?download=1
Download link found: /records/4656548/files/m4_daily_dataset.zip?download=1
Download link found: /records/4656589/files/m4_hourly_dataset.zip?

In [6]:
def download_and_unzip(url, extract_to='.'):
    # Download the file from `url` and save it locally under `file_name`:
    local_zip_file = 'temp.zip'
    
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_zip_file, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    
    # Unzip the file
    with zipfile.ZipFile(local_zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    
    # Optionally remove the zip file after extraction
    os.remove(local_zip_file)

In [18]:
df[df['Domain'] == 'Economic']

,index,Dataset,Domain,No: of Series,Min. Length,Max. Length,Competition,Multivariate,Download,Source,Download Links,download_link
23,9,Bitcoin,Economic,18,2659,4581,No,No,W Missing,Curated by us,https://zenodo.org/record/5121965,https://zenodo.org/records/5121965/files/bitco...
24,9,Bitcoin,Economic,18,2659,4581,No,No,W/O Missing,Curated by us,https://zenodo.org/record/5122101,https://zenodo.org/records/5122101/files/bitco...
43,19,FRED-MD,Economic,107,728,728,No,Yes,Monthly,"McCracken and Ng, 2016",https://zenodo.org/record/4654833,https://zenodo.org/records/4654833/files/fred_...


In [19]:
download_links = df[df['Domain'] == 'Economic']['download_link'].tolist()

In [20]:
for data_zip_path in download_links:
    download_and_unzip(data_zip_path, extract_to='./files')

In [21]:
def list_tsf_files(folder_path):
    # List all files in the given folder
    files = os.listdir(folder_path)
    # Filter out the files that end with .tsf
    tsf_files = [folder_path + '/' + f for f in files if f.endswith('.tsf')]
    return tsf_files

In [22]:
files = list_tsf_files('./files')

In [23]:
# Code snippet taking from https://github.com/rakshitha123/TSForecasting/blob/master/utils/data_loader.py
from datetime import datetime
from distutils.util import strtobool

import pandas as pd


# Converts the contents in a .tsf file into a dataframe and returns it along with other meta-data of the dataset: frequency, horizon, whether the dataset contains missing values and whether the series have equal lengths
#
# Parameters
# full_file_path_and_name - complete .tsf file path
# replace_missing_vals_with - a term to indicate the missing values in series in the returning dataframe
# value_column_name - Any name that is preferred to have as the name of the column containing series values in the returning dataframe
def convert_tsf_to_dataframe(
    full_file_path_and_name,
    replace_missing_vals_with="NaN",
    value_column_name="series_value",
):
    col_names = []
    col_types = []
    all_data = {}
    line_count = 0
    frequency = None
    forecast_horizon = None
    contain_missing_values = None
    contain_equal_length = None
    found_data_tag = False
    found_data_section = False
    started_reading_data_section = False

    with open(full_file_path_and_name, "r", encoding="cp1252") as file:
        for line in file:
            # Strip white space from start/end of line
            line = line.strip()

            if line:
                if line.startswith("@"):  # Read meta-data
                    if not line.startswith("@data"):
                        line_content = line.split(" ")
                        if line.startswith("@attribute"):
                            if (
                                len(line_content) != 3
                            ):  # Attributes have both name and type
                                raise Exception("Invalid meta-data specification.")

                            col_names.append(line_content[1])
                            col_types.append(line_content[2])
                        else:
                            if (
                                len(line_content) != 2
                            ):  # Other meta-data have only values
                                raise Exception("Invalid meta-data specification.")

                            if line.startswith("@frequency"):
                                frequency = line_content[1]
                            elif line.startswith("@horizon"):
                                forecast_horizon = int(line_content[1])
                            elif line.startswith("@missing"):
                                contain_missing_values = bool(
                                    strtobool(line_content[1])
                                )
                            elif line.startswith("@equallength"):
                                contain_equal_length = bool(strtobool(line_content[1]))

                    else:
                        if len(col_names) == 0:
                            raise Exception(
                                "Missing attribute section. Attribute section must come before data."
                            )

                        found_data_tag = True
                elif not line.startswith("#"):
                    if len(col_names) == 0:
                        raise Exception(
                            "Missing attribute section. Attribute section must come before data."
                        )
                    elif not found_data_tag:
                        raise Exception("Missing @data tag.")
                    else:
                        if not started_reading_data_section:
                            started_reading_data_section = True
                            found_data_section = True
                            all_series = []

                            for col in col_names:
                                all_data[col] = []

                        full_info = line.split(":")

                        if len(full_info) != (len(col_names) + 1):
                            raise Exception("Missing attributes/values in series.")

                        series = full_info[len(full_info) - 1]
                        series = series.split(",")

                        if len(series) == 0:
                            raise Exception(
                                "A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series. Missing values should be indicated with ? symbol"
                            )

                        numeric_series = []

                        for val in series:
                            if val == "?":
                                numeric_series.append(replace_missing_vals_with)
                            else:
                                numeric_series.append(float(val))

                        if numeric_series.count(replace_missing_vals_with) == len(
                            numeric_series
                        ):
                            raise Exception(
                                "All series values are missing. A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series."
                            )

                        all_series.append(pd.Series(numeric_series).array)

                        for i in range(len(col_names)):
                            att_val = None
                            if col_types[i] == "numeric":
                                att_val = int(full_info[i])
                            elif col_types[i] == "string":
                                att_val = str(full_info[i])
                            elif col_types[i] == "date":
                                att_val = datetime.strptime(
                                    full_info[i], "%Y-%m-%d %H-%M-%S"
                                )
                            else:
                                raise Exception(
                                    "Invalid attribute type."
                                )  # Currently, the code supports only numeric, string and date types. Extend this as required.

                            if att_val is None:
                                raise Exception("Invalid attribute value.")
                            else:
                                all_data[col_names[i]].append(att_val)

                line_count = line_count + 1

        if line_count == 0:
            raise Exception("Empty file.")
        if len(col_names) == 0:
            raise Exception("Missing attribute section.")
        if not found_data_section:
            raise Exception("Missing series information under data section.")

        all_data[value_column_name] = all_series
        loaded_data = pd.DataFrame(all_data)

        return (
            loaded_data,
            frequency,
            forecast_horizon,
            contain_missing_values,
            contain_equal_length,
        )


# Example of usage
# loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe("TSForecasting/tsf_data/sample.tsf")

# print(loaded_data)
# print(frequency)
# print(forecast_horizon)
# print(contain_missing_values)
# print(contain_equal_length)

In [24]:
def convert_to_pandas_frequency(frequency_str):
    # Dictionary to map units to pandas frequency codes
    unit_map = {
        'second': 'S',
        'seconds': 'S',
        'minute': 'T',
        'minutes': 'T',
        'hour': 'H',
        'hours': 'H',
        'hourly': 'H',
        'day': 'D',
        'days': 'D',
        'daily': 'D',
        'week': 'W',
        'weeks': 'W',
        'weekly': 'W',
        'month': 'M',
        'months': 'M',
        'monthly': 'M',
        'year': 'A',
        'years': 'A',
        'yearly': 'A',
        'quarter': 'Q',
        'quarters': 'Q',
        'quarterly': 'Q'
    }
    
    # Split the input string to get the number and unit
    parts = frequency_str.split('_')
    if len(parts) == 1:
        number = 1
        unit = parts[0]
        #raise ValueError("Input format should be '<number>_<unit>'")
    elif len(parts) ==2:
        number, unit = parts
        # Dict"ionary to map each unit to its half-unit equivalent
        half_unit_map = {
            'A': ('Q', 2),  # Half of a year is 2 quarters
            'Q': ('M', 2),  # Half of a quarter is 2 months
            'M': ('D', 15),  # Approximate half of a month is 15 days
            'W': ('D', 3.5),  # Half of a week is 3.5 days
            'D': ('H', 12),  # Half of a day is 12 hours
            'H': ('T', 30),  # Half of an hour is 30 minutes
            'T': ('S', 30)  # Half of a minute is 30 seconds
        }
    
        if number == 'half':
            # Get the unit key from the unit map
            unit_key = unit_map[unit]
            
            # Get the half-unit equivalent from the half_unit_map
            if unit_key in half_unit_map:
                half_unit, half_number = half_unit_map[unit_key]
                pandas_frequency = f"{int(half_number)}{half_unit}"
                return pandas_frequency
            else:
                raise ValueError(f"Half unit mapping not available for unit '{unit}'")
        else:
            try:
                number = int(number)
            except ValueError:
                raise ValueError("The number part of the input should be an integer")
    else:
        raise ValueError("Input format should be '<number>_<unit>' Maybe update unit_map")
    

    
    if unit not in unit_map:
        raise ValueError(f"Unit '{unit}' is not recognized")
    
    # Convert to pandas frequency string
    pandas_frequency = f"{number}{unit_map[unit]}"
    return pandas_frequency

In [25]:
def generate_timestamps(start_timestamp, frequency, values):
    # Generate a range of timestamps
    timestamps = pd.date_range(start=start_timestamp, periods=len(values), freq=convert_to_pandas_frequency(frequency))
    
    # Create a DataFrame with the values and the generated timestamps
    df = pd.DataFrame({'Timestamp': timestamps, 'Value': values})
    
    return df

In [26]:
df_list = []
for file in files:
    name = file.split('/')[-1].split('.tsf')[0]
    loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe(file)
    for index, row in loaded_data.iterrows():
        serie_name = name + '_' + row['series_name']
        serie = generate_timestamps(row['start_timestamp'], frequency, row['series_value'])
        df_list.append(serie)
        serie.to_csv("./files/" + serie_name + ".csv")
    os.remove(file)


/var/folders/np/tdfp6mp11c32fmn2ds87h7540000gn/T/ipykernel_56088/3065550573.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  timestamps = pd.date_range(start=start_timestamp, periods=len(values), freq=convert_to_pandas_frequency(frequency))


In [29]:
files

['./files/fred_md_dataset.tsf',
 './files/bitcoin_dataset_without_missing_values.tsf',
 './files/bitcoin_dataset_with_missing_values.tsf']

In [28]:
len(df_list)

143

In [102]:
foo = pd.read_csv("./files/m1_quarterly_dataset.tsf", sep='\t',encoding='cp1252',header=0)

In [113]:
foo[(~foo['# Dataset Information'].str.startswith('#'))&(~foo['# Dataset Information'].str.startswith(''))]

,# Dataset Information
16,"T1:1975-10-01 00-00-00:0.54,0.78,0.82,0.46,0.5..."
17,"T2:1974-10-01 00-00-00:97,95,95,94,93,90,90,90..."
18,"T3:1975-10-01 00-00-00:94,132,131,127,77,103,1..."
19,"T4:1977-04-01 00-00-00:9366.37,9169.56,7776.84..."
20,"T5:1978-04-01 00-00-00:503674,549712,547329,55..."
...,...
214,"T199:1975-10-01 00-00-00:1184,1196,1209,1216,1..."
215,"T200:1975-10-01 00-00-00:71,50.9,40,43,51.84,4..."
216,"T201:1975-10-01 00-00-00:40.8,36.9,44.2,62.1,5..."
217,"T202:1975-10-01 00-00-00:659,660,661,673,686,6..."
